In [ ]:

!apt-get install -y aria2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 amd64 1.18.1-1ubuntu0.22.04.3 [45.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libaria2-0 amd64 1.36.0-1 [1,086 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aria2 amd64 1.36.0-1 [381 kB]
Fetched 1,513 kB in 2s (675 kB/s)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123586 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu

In [ ]:
!apt-get install -y aria2

from google.colab import drive
import os
from urllib.parse import urlparse, unquote
import time
import threading
import subprocess
from tqdm import tqdm
import requests

# ANSI escape codes for bold text
BOLD = '\033[1m'
END = '\033[0m'

drive.mount('/content/drive')

# Define the destination folder in Google Drive
destination_folder = '/content/drive/MyDrive/Uploader/'

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

def format_file_size(size):
    """Helper function to format file size."""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024:
            return f"{size:.2f} {unit}"
        size /= 1024

def download_file_with_aria2(url, file_path, cancel_event, progress_bar):
    """Function to download a file using aria2 with cancel option and progress bar."""
    aria2_process = subprocess.Popen(
        ["aria2c", "--summary-interval=1", "--download-result=hide", "-x16", "-s16", "-j5", url, "-d", os.path.dirname(file_path), "-o", os.path.basename(file_path)],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    while True:
        output = aria2_process.stdout.readline()
        if output == '' and aria2_process.poll() is not None:
            break
        if output:
            if cancel_event.is_set():
                aria2_process.terminate()
                print(f"{BOLD}\nDownload canceled.{END}")
                return False
            # Parse the line for progress updates
            if output.startswith("[#"):
                parts = output.split()
                if len(parts) > 3:
                    try:
                        downloaded_size = parts[2]
                        if downloaded_size.endswith('M'):
                            progress_bar.n = float(downloaded_size[:-1]) * 1024 * 1024
                        elif downloaded_size.endswith('K'):
                            progress_bar.n = float(downloaded_size[:-1]) * 1024
                        elif downloaded_size.endswith('G'):
                            progress_bar.n = float(downloaded_size[:-1]) * 1024 * 1024 * 1024
                        progress_bar.refresh()
                    except ValueError:
                        continue

    return aria2_process.returncode == 0

def cancel_download(cancel_event):
    input(f"{BOLD}Type 'cancel' to cancel the downloading...{END}")
    cancel_event.set()

try:
    while True:
        try:
            print(f"{BOLD}\n\n\nWelcome to the Google Drive Uploader 🙏🏻 \n\n{END}")
            print("Type 'exit' to quit the program.\n")
            user_input = input("Enter the download link 🔗: ").strip()
        except KeyboardInterrupt:
            print(f"{BOLD}\nKeyboard interrupt detected. Exiting the program.{END}")
            break

        if user_input.lower() == 'exit':
            print(f"{BOLD}Exiting the program.{END}")
            break

        try:
            response = requests.head(user_input, allow_redirects=True)
            response.raise_for_status()
        except requests.exceptions.MissingSchema:
            print(f"{BOLD}Invalid URL. Please enter a valid download link.\n{END}")
            continue
        except requests.exceptions.RequestException as e:
            print(f"{BOLD}Error fetching the URL: {e}. Please try again.\n{END}")
            continue

        file_size = int(response.headers.get('content-length', 0))
        file_size_str = format_file_size(file_size)

        parsed_url = urlparse(user_input)
        file_name = unquote(os.path.basename(parsed_url.path))

        print(f"{BOLD}\nThe file '{file_name}' is {file_size_str} in size.{END}")
        rename_choice = input("Do you want to rename the file? (yes/no): ").strip().lower()
        if rename_choice == 'yes':
            file_name = input("Enter the new file name with extension (e.g., myfile.pdf): ").strip()

        file_path = os.path.join(destination_folder, file_name)

        start_time = time.time()

        cancel_event = threading.Event()
        cancel_thread = threading.Thread(target=cancel_download, args=(cancel_event,))
        cancel_thread.start()

        progress_bar = tqdm(total=file_size, unit='B', unit_scale=True, leave=True)

        download_success = download_file_with_aria2(user_input, file_path, cancel_event, progress_bar)

        progress_bar.close()
        cancel_event.set()  # Ensure the event is set to close the cancel thread

        if not download_success:
            if os.path.exists(file_path):
                os.remove(file_path)
            print("File downloading was canceled and the incomplete file has been removed.")
        else:
            end_time = time.time()
            total_time = end_time - start_time

            print()
            print(f"{BOLD}File name:{END} {file_name}")
            print(f"{BOLD}File size:{END} {file_size_str}")
            print(f"{BOLD}Total Time:{END} {total_time:.2f} seconds")
            print(f"{BOLD}Saved location:{END} {file_path}")
            print(f"{BOLD}File '{file_name}' downloaded successfully to '{destination_folder}'.{END}\n")

        cancel_thread.join()  # Ensure the cancel thread terminates before continuing

except Exception as e:
    print(f"{BOLD}An unexpected error occurred:{END} {str(e)}")
    print(f"{BOLD}Exiting the program.{END}")

print(f"{BOLD}Exiting the program.{END}")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 amd64 1.18.1-1ubuntu0.22.04.3 [45.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libaria2-0 amd64 1.36.0-1 [1,086 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aria2 amd64 1.36.0-1 [381 kB]
Fetched 1,513 kB in 1s (1,715 kB/s)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubun